## Q-Learning Tutorial
I wanted to understand reinforcement learning a little better for a hackathon project at work. One approach I remember learning about in a machine learning class in college is called Q-learning. I found a blog post that briefly goes over this approach from a very high level and so to understand it better I decided to code up the example in the blog. The blog is found at: https://towardsdatascience.com/q-bay-explaining-q-learning-with-simulated-auctions-f85bac990c60

As the blog mentions there are two matrices or tables to be concerned about when dealing with Q-learning. The first is the R table and the second is the Q table. 

The R table is the table that contains the rewards of going certain places.

So in the example given in the blog post we want to teach the computer to go to a certain subway station given any other starting point. I've included the reward table from the blog below:

|Stations |Arsenal       | Finsbusry Park | Manor House  |  Seven Sisters | Stamford Hill | Tottenham Hale |
|:--- |:------------:|:-------------: | :-----:      |:-------------: |:-------------:| :-----:        |
    |**Arsenal** |0      | 0 | - |-      | - | - |
|**Finsbusry Park** |0      | 0 | 0 |0      | - | - |
|**Manor House**|-      | 0 | 0 |-      | - | - |
|**Seven Sisters** |-      | 0 | - |0      | 0 | 100 |
|**Stamford Hill** |-      | - | - |0      | 0 | - |
|**Tottenham Hale** |-      | - | - |0      | - | 100 |

Imagine the rows representing the states (where you are at now) and the columns representing the actions (where you can go.) In this case Tottenham Hale is where we want to end up. Lets pretend that the only way to get there is through the Seven Sisters train station, and if we get to Tottenham Hale we want to stay there. Thus the values 100 in both those positions.

The Q matrix will be the exact same format except everything will be initialized to 0. I put both of these tables into a pandas data frame below.

In [1]:
import pandas as pd
import numpy as np

In [2]:
Q = np.zeros((6,6))
R = np.zeros((6,6))

# Fill in the rewards
R[3,5] = 100
R[5,5] = 100

# Fill in state action pairs with null values if it is not possible to go from state to action
R[2:,0] = np.nan
R[4:,1] = np.nan
R[0,2] = np.nan
R[3:,2] = np.nan
R[0,3] = np.nan
R[2,3] = np.nan
R[:3,4] = np.nan
R[5,4] = np.nan
R[:3,5] = np.nan
R[4,5] = np.nan

Q_table = pd.DataFrame(Q)
R_table = pd.DataFrame(R)

In [3]:
R_table

,0,1,2,3,4,5
0,0.0,0.0,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,NaN,NaN
2,NaN,0.0,0.0,NaN,NaN,NaN
3,NaN,0.0,NaN,0.0,0.0,100.0
4,NaN,NaN,NaN,0.0,0.0,NaN
5,NaN,NaN,NaN,0.0,NaN,100.0


In [4]:
Q_table

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0


Now that both tables are initialized the first step is to pick a random starting point. In order to follow the example in the blog say we choose random state "Stamford Hill" which is state 4

In [5]:
start_state = 4

Given our starting point now we need to determine which action to take. Since we are initializing this "episode" we choose a random action out of the available actions for the given state. Say we randomly choose "Seven Sisters" to follow the example.

In [6]:
start_action = 3

In [7]:
%%latex
Now we use the q-learning equation to update the matrix Q:

\begin{equation}
Q(s_t, at) = (1 - \alpha) Q(s_t, a_t) + \alpha[R(s_t, a_t) + \gamma max{Q(s_{t+1}, a_{t+1})}]
\end{equation}



<IPython.core.display.Latex object>

In [8]:
alpha = .1 # learning rate
gamma = .1 # gamma is the discount factor

Q_table.iloc[start_state, start_action] = (1 - alpha)*Q_table.iloc[start_state, start_action] + alpha*R_table.iloc[start_state, start_action] + gamma*Q_table.iloc[start_action,:].idxmax()

In [9]:
Q_table

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0


This should have updated the Q matrix to 0, so no new information. We are now at "Seven Sisters" and can either go to "Finsbury Park", "Seven Sisters" (stay in the same state), "Stamford Hill", or "Tottenham Hill". The Q matrix provides no information with where to go so we choose a random action again; to follow the example we choose Tottenham Hill.

In [10]:
start_state = start_action
start_action = 5

Update Q matrix 

In [11]:
Q_table.iloc[start_state, start_action] = (1 - alpha)*Q_table.iloc[start_state, start_action] + alpha*R_table.iloc[start_state, start_action] + gamma*Q_table.iloc[start_action,:].idxmax()



In [12]:
Q_table

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,10.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0


Notice that Q(3,5) is 10 since the reward has propogated through. Since we reached the end goal ("Tottenham") we start the process over which I do in a for loop below.

In [13]:
import random

In [14]:
done=False
for i in xrange(100):
    # If we are starting the episode over (we just ran into the final destination)
    if done==True:
        start_state = random.randint(0,5)
        print("Start state: {0}".format(start_state))
        # Now all actions are available at the given state, chose out of the ones that are available
        indices = R_table.iloc[start_state,:].index[R_table.iloc[start_state,:].notnull()].values
        start_action = random.choice(indices)
        print("Start action: {0}".format(start_action))
    
    # Update Q matrix
    Q_table.iloc[start_state, start_action] = (1 - alpha)*Q_table.iloc[start_state, start_action] + alpha*R_table.iloc[start_state, start_action] + gamma*Q_table.iloc[start_action,:].idxmax()
    if start_action==5:
        done=True
    else:
        start_state = start_action
        indices = R_table.iloc[start_state,:].index[R_table.iloc[start_state,:].notnull()].values
        start_action = random.choice(indices)
        print("Start action: {0}".format(start_action))
    print(Q_table)

     0    1    2    3    4     5
0  0.0  0.0  0.0  0.0  0.0   0.0
1  0.0  0.0  0.0  0.0  0.0   0.0
2  0.0  0.0  0.0  0.0  0.0   0.0
3  0.0  0.0  0.0  0.0  0.0  19.0
4  0.0  0.0  0.0  0.0  0.0   0.0
5  0.0  0.0  0.0  0.0  0.0   0.0
Start state: 3
Start action: 1
Start action: 0
     0    1    2    3    4     5
0  0.0  0.0  0.0  0.0  0.0   0.0
1  0.0  0.0  0.0  0.0  0.0   0.0
2  0.0  0.0  0.0  0.0  0.0   0.0
3  0.0  0.0  0.0  0.0  0.0  19.0
4  0.0  0.0  0.0  0.0  0.0   0.0
5  0.0  0.0  0.0  0.0  0.0   0.0
Start state: 0
Start action: 1
Start action: 0
     0    1    2    3    4     5
0  0.0  0.0  0.0  0.0  0.0   0.0
1  0.0  0.0  0.0  0.0  0.0   0.0
2  0.0  0.0  0.0  0.0  0.0   0.0
3  0.0  0.0  0.0  0.0  0.0  19.0
4  0.0  0.0  0.0  0.0  0.0   0.0
5  0.0  0.0  0.0  0.0  0.0   0.0
Start state: 1
Start action: 2
Start action: 1
     0    1    2    3    4     5
0  0.0  0.0  0.0  0.0  0.0   0.0
1  0.0  0.0  0.0  0.0  0.0   0.0
2  0.0  0.0  0.0  0.0  0.0   0.0
3  0.0  0.0  0.0  0.0  0.0  19.0
4

5  0.0  0.00  0.0  0.950  0.0  28.293
Start state: 3
Start action: 5
     0     1    2      3    4       5
0  0.0  0.57  0.0  0.000  0.0   0.000
1  0.0  0.00  0.0  1.355  0.0   0.000
2  0.0  0.00  0.0  0.000  0.0   0.000
3  0.0  0.30  0.0  0.500  0.0  27.600
4  0.0  0.00  0.0  0.000  0.0   0.000
5  0.0  0.00  0.0  0.950  0.0  28.293
Start state: 0
Start action: 1
Start action: 1
     0      1    2      3    4       5
0  0.0  0.813  0.0  0.000  0.0   0.000
1  0.0  0.000  0.0  1.355  0.0   0.000
2  0.0  0.000  0.0  0.000  0.0   0.000
3  0.0  0.300  0.0  0.500  0.0  27.600
4  0.0  0.000  0.0  0.000  0.0   0.000
5  0.0  0.000  0.0  0.950  0.0  28.293
Start state: 3
Start action: 3
Start action: 5
     0      1    2      3    4       5
0  0.0  0.813  0.0  0.000  0.0   0.000
1  0.0  0.000  0.0  1.355  0.0   0.000
2  0.0  0.000  0.0  0.000  0.0   0.000
3  0.0  0.300  0.0  0.950  0.0  27.600
4  0.0  0.000  0.0  0.000  0.0   0.000
5  0.0  0.000  0.0  0.950  0.0  28.293
Start state: 1
Start acti

        0       1     2        3    4          5
0  0.3439  1.0317  0.00  0.00000  0.0   0.000000
1  0.1900  0.0000  0.19  1.35500  0.0   0.000000
2  0.0000  0.5700  0.00  0.00000  0.0   0.000000
3  0.0000  1.0317  0.00  2.04755  0.0  27.600000
4  0.0000  0.0000  0.00  0.50000  0.0   0.000000
5  0.0000  0.0000  0.00  2.04755  0.0  49.080597
Start state: 5
Start action: 3
Start action: 1
        0       1     2         3    4          5
0  0.3439  1.0317  0.00  0.000000  0.0   0.000000
1  0.1900  0.0000  0.19  1.355000  0.0   0.000000
2  0.0000  0.5700  0.00  0.000000  0.0   0.000000
3  0.0000  1.0317  0.00  2.047550  0.0  27.600000
4  0.0000  0.0000  0.00  0.500000  0.0   0.000000
5  0.0000  0.0000  0.00  2.342795  0.0  49.080597
Start state: 0
Start action: 1
Start action: 3
        0        1     2         3    4          5
0  0.3439  1.22853  0.00  0.000000  0.0   0.000000
1  0.1900  0.00000  0.19  1.355000  0.0   0.000000
2  0.0000  0.57000  0.00  0.000000  0.0   0.000000
3  0.0000

5  0.000000  0.000000  0.00  3.256608  0.00  75.282137
Start state: 5
Start action: 5
          0         1     2         3     4          5
0  0.569533  1.405677  0.00  0.000000  0.00   0.000000
1  0.271000  0.570000  0.19  1.719500  0.00   0.000000
2  0.000000  1.228530  0.00  0.000000  0.00   0.000000
3  0.000000  1.228530  0.00  2.608516  0.00  35.340000
4  0.000000  0.000000  0.00  0.950000  0.57   0.000000
5  0.000000  0.000000  0.00  3.256608  0.00  78.253923
Start state: 3
Start action: 1
Start action: 2
          0         1     2         3     4          5
0  0.569533  1.405677  0.00  0.000000  0.00   0.000000
1  0.271000  0.570000  0.19  1.719500  0.00   0.000000
2  0.000000  1.228530  0.00  0.000000  0.00   0.000000
3  0.000000  1.405677  0.00  2.608516  0.00  35.340000
4  0.000000  0.000000  0.00  0.950000  0.57   0.000000
5  0.000000  0.000000  0.00  3.256608  0.00  78.253923
Start state: 5
Start action: 5
          0         1     2         3     4          5
0  0.569533